# Texterfassung Steuerzentrale für Yijing

## 1. Imports und Konfiguration

In [ ]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any

import google.generativeai as genai
import pandas as pd
from IPython.display import Markdown, display

### API-Konfiguration

In [ ]:
# Konfiguration des API-Schlüssels
api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("Die Umgebungsvariable 'API_KEY' ist nicht gesetzt.")
genai.configure(api_key=api_key)

### Modellwahl

In [ ]:
# Auswahl des Modells
model_speed = {
    'dumb': 'gemini-1.5-flash-8b',
    'fast': 'gemini-1.5-flash-latest',
    'clever': 'gemini-1.5-pro-latest',
    'experimental': 'gemini-exp-1121',
}
model_type = os.getenv("MODEL_TYPE", model_speed['experimental'])
print("Verwendetes Modell:", model_type)

## 2. Datenimport

### Yijing Text einlesen

In [ ]:
yijing_txt_path = Path('yijing/resources/yijing.txt')

# Funktion zum Lesen des Yijing-Texts
def read_yijing_txt(path: Path = yijing_txt_path) -> str:
    with path.open('r', encoding='utf-8') as f:
        return f.read()

# Laden der Daten
yijing_txt = read_yijing_txt()
print("Yijing-Text erfolgreich geladen.")

### Erste Einblicke in die Daten

In [ ]:
# Anzeige der ersten 500 Zeichen des Yijing-Texts
print(yijing_txt[:500])

## 3. Datenverarbeitung

### Strukturierter JSON-Text des ersten Kapitels

In [ ]:
yijing_processed = """
{
  "hexagram": {
    "name": "GUAI / DER DURCHBRUCH",
    "subtitle": "Die Entschlossenheit",
    "trigrams": {
      "above": {
        "name": "Dui",
        "attributes": "das Heitere, der See"
      },
      "below": {
        "name": "Kien",
        "attributes": "das Schöpferische, der Himmel"
      }
    },
    "meaning": {
      "description": "Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung...",
      "season": "dritter Monat (April-Mai)"
    }
  },
  "judgment": {
    "description": "Der Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen...",
    "analysis": [
      "Leidenschaft und Vernunft können nicht zusammen bestehen...",
      "Entschlossenheit muß auf Stärke und Freundlichkeit beruhen..."
    ]
  },
  "image": {
    "description": "Der See ist an den Himmel emporgestiegen...",
    "lesson": "Der Edle spendet Reichtum nach unten hin...",
    "warning": "Sammeln führt zu Zerstreuen..."
  },
  "lines": [
    {
      "position": "Anfangs eine",
      "text": "Mächtig in den vorwärtsschreitenden Zehen...",
      "interpretation": "Zu Beginn ist entschlossenes Voranschreiten schwierig..."
    }
    // Weitere Linien können hier hinzugefügt werden
  ]
}
"""

# Umwandlung des JSON-Textes in ein Python-Dictionary
yijing_data = json.loads(yijing_processed)
print("Erstes Kapitel erfolgreich verarbeitet.")

## 4. Funktionen zur Verarbeitung eines Hexagramms

In [ ]:
def process_hexagram(text: str, model: genai.GenerativeModel) -> Dict[str, Any]:
    """
    Verarbeitet einen Hexagramm-Text und gibt strukturierte Daten zurück.
    """
    prompt = HEXAGRAM_PROMPT + text
    result = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=None  # Verwenden Sie None, wenn kein Schema benötigt wird
        ),
    )
    return result.candidates[0].content

## 5. Initialisierung des Modells

### Systemanweisung für das Modell

In [ ]:
HEXAGRAM_PROMPT = """
Du bist ein I Ging-Experte mit der Aufgabe, Hexagramm-Texte zu analysieren und in ein spezifisches JSON-Format zu überführen.

Wandle den Eingabetext in folgendes Format um:

{
  "hexagram": {
    "name": "Name des Hexagramms",
    "subtitle": "Untertitel",
    "trigrams": {
      "above": {"name": "", "attributes": ""},
      "below": {"name": "", "attributes": ""}
    },
    "meaning": {"description": "", "season": ""}
  },
  "judgment": {"description": "", "analysis": [""]},
  "image": {"description": "", "lesson": "", "warning": ""},
  "lines": [
    {"position": "", "text": "", "interpretation": ""}
    // Weitere Linien
  ]
}

Analysiere nun den folgenden Hexagramm-Text und gib ihn im spezifizierten JSON-Format zurück:
"""

In [ ]:
instruction = HEXAGRAM_PROMPT

In [ ]:
# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

# Chat-Instanz starten
chat = model.start_chat()
print("Modell und Chat-Instanz erfolgreich initialisiert.")

## 6. Verarbeitung und Export

### Hexagramm verarbeiten und anzeigen

In [ ]:
def process_hexagram_text(hexagram_text: str) -> Dict[str, Any]:
    """
    Verarbeitet einen Hexagramm-Text und gibt strukturierte Daten zurück.

    Args:
        hexagram_text (str): Der Hexagramm-Text.

    Returns:
        dict: Strukturierte Daten des Hexagramms.
    """
    result = chat.send_message(hexagram_text)

    # Überprüfen, ob die Antwort Teile enthält
    if hasattr(result, 'parts') and len(result.parts) > 0:
        json_text = result.parts[0].text
        # Entfernen von Codeblöcken, falls vorhanden
        formatted_text = json_text.strip('```json\n').strip('```')
        
        try:
            # Laden des JSON in ein Python-Dictionary
            data = json.loads(formatted_text)
            display(Markdown("### Verarbeitetes Hexagramm"))
            display(json.dumps(data, indent=4, ensure_ascii=False))
            return data
        except json.JSONDecodeError:
            print("Fehler beim Parsen des JSON.")
    else:
        print("Keine Antwortteile gefunden.")
    
    return {}

In [ ]:
# Beispielhafte Nutzung mit dem ersten Kapitel
first_chapter_text = yijing_chapters[0] if 'yijing_chapters' in locals() else yijing_txt[:1000]
extracted_hexagram = process_hexagram_text(first_chapter_text)

### Speicherung der Daten als JSON

In [ ]:
def save_hexagram_json(hexagram_data: Dict[str, Any], output_dir: Path, filename: str) -> None:
    """
    Speichert die Hexagramm-Daten als JSON-Datei.

    Args:
        hexagram_data (dict): Strukturierte Hexagramm-Daten.
        output_dir (Path): Verzeichnis zum Speichern der JSON-Dateien.
        filename (str): Name der JSON-Datei.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / filename
    with output_file.open('w', encoding='utf-8') as f:
        json.dump(hexagram_data, f, ensure_ascii=False, indent=4)
    print(f"Daten gespeichert unter {output_file}")

In [ ]:
# Speichern des extrahierten Hexagramms
if extracted_hexagram:
    save_hexagram_json(extracted_hexagram, Path('export/hexagram_json'), 'hexagram_01.json')

### Export der Linien in ein DataFrame

In [ ]:
def export_lines_to_dataframe(hexagram_data: Dict[str, Any]) -> pd.DataFrame:
    """
    Exportiert die Linien eines Hexagramms in ein pandas DataFrame.

    Args:
        hexagram_data (dict): Strukturierte Hexagramm-Daten.

    Returns:
        pd.DataFrame: DataFrame mit den Linieninformationen.
    """
    lines = hexagram_data.get('lines', [])
    lines_df = pd.DataFrame(lines)
    return lines_df

# Erstellen des DataFrames
lines_df = export_lines_to_dataframe(extracted_hexagram)
display(lines_df)

## 7. Zusätzliche Analysen (Optional)

### Beispiel: Analyse der Interpretationen

In [ ]:
if not lines_df.empty:
    from collections import Counter
    import re
    
    # Zusammenführen aller Interpretationen
    all_interpretations = ' '.join(lines_df['interpretation'].dropna().tolist())
    
    # Extrahieren von Wörtern
    words = re.findall(r'\w+', all_interpretations.lower())
    word_counts = Counter(words)
    
    # Anzeige der häufigsten Wörter
    common_words = word_counts.most_common(10)
    common_df = pd.DataFrame(common_words, columns=['Wort', 'Häufigkeit'])
    display(Markdown("### Häufigste Wörter in den Interpretationen"))
    display(common_df)

# Ende des Notebooks